In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# from sklearn import ?
# from sklearn.metrics import ?

<br>

## 1. Preparing dataset (2번부터 실습 진행)

In [2]:
data_df = pd.read_csv('titanic.csv')
data_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Data info

- **PassengerId** : Unique ID of passenger
- **Survived** : 0 = No, 1 = Yes
- **pclass** : Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
- **sibsp** : # of siblings & spouses aboard the Titanic
- **parch** : # of parents / children aboard the Titanic
- **ticket** : Ticket number
- **cabin** : Cabin number
- **embarked** : Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

In [3]:
y_data = data_df[['Survived']]
y_data.head(3)

,Survived
0,0
1,1
2,1


In [4]:
del data_df['Survived']
x_data = data_df.copy()
x_data.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


<br>

## 2. Feature engineering & Feature selection

#### 시도해볼 수 있는 전략들

- 불필요한 열이나 예측에 방해가 되는 열은 아예 지우기 (ex. PassengerId)
- 결측치 채우기 
- Text로 되어있는 Category(Factor)는 숫자로 바꿔주기 (ex. Male/Female -> 0/1)
- 실수 범위를 구간 범위로 바꿔주기 
- 필요한 경우 기존 열을 바탕으로 새로운 열을 계산해 추가하기

## 원하는 전처리

### Pclass, Sex(one-hot encoding)
### Age(Min-Max? Standardizatin?)
### Companion(SibSp+parch, 같은 티켓)
### Fare(고민?)
### Embarked(고민??)
### Cabin(one hot?)

In [5]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [6]:
x_data['Embarked']

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

## [AGE]열 결측치 -> 중앙값
---

In [7]:
# 기본적인 결측치 채우는 방법
mean_age = data_df['Age'].median(skipna=True)
data_df['Age'] = data_df['Age'].fillna(mean_age) # or .fillna(int/float/str 값)

In [8]:
data_df['Age'] = data_df['Age'] /10

## ['Sex']열 One-hot encoding

In [9]:
data_df['Sex'] = pd.get_dummies(data_df['Sex']) # 남자 0, 여자 1

📌 배우자 형제랑 같이 있으면 같이 탈출 가능성 높아질수도?, 부모자식이면 느려질 수도? 

## ['Companion'] 열 만들기
['SibSp'] 배우자, 형제 + ['Parch'] 부모 자식  
['Ticket'] 중복시 친구?? 일행 가능성?

In [10]:
# x_data['Companion'] = x_data['SibSp'] + x_data['Parch']
# del x_data['SibSp']
# del x_data['Parch']

In [11]:
count_dict

NameError: name 'count_dict' is not defined

In [ ]:
#티켓 중복 숫자 세기
count_dict = dict(data_df['Ticket'].value_counts())
#SibSp, Parch 둘다 0 이고 티켓 넘버 같은 것도 없을 경우 일행 없는 것으로 간주하고 0, 나머지는 1(일행있음)
data_df['Companion'] = [0 if data_df['SibSp'][x] == 0 and data_df['Parch'][x] == 0 and count_dict[data_df['Ticket'][x]] == 1 else 1 for x in data_df.index ]

In [ ]:
data_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Companion
0,1,3,"Braund, Mr. Owen Harris",0,2.2,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,3.8,1,0,PC 17599,71.2833,C85,C,1
2,3,3,"Heikkinen, Miss. Laina",1,2.6,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,3.5,1,0,113803,53.1000,C123,S,1
4,5,3,"Allen, Mr. William Henry",0,3.5,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",0,2.7,0,0,211536,13.0000,NaN,S,0
887,888,1,"Graham, Miss. Margaret Edith",1,1.9,0,0,112053,30.0000,B42,S,0
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,2.8,1,2,W./C. 6607,23.4500,NaN,S,1
889,890,1,"Behr, Mr. Karl Howell",0,2.6,0,0,111369,30.0000,C148,C,0


## ['Fare']열 자릿수 맞추기

In [ ]:
data_df['Cabin']

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object

In [ ]:
data_df['Cabin'] = ~data_df['Cabin'].isnull()

In [ ]:
data_df['Cabin'] = data_df['Cabin'].astype(int)

## 필요한 열만 뽑아내는 과정

In [ ]:
x_data = data_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]

x_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,0,2.2,1,0,7.2500,0,S
1,1,1,3.8,1,0,71.2833,1,C
2,3,1,2.6,0,0,7.9250,0,S
3,1,1,3.5,1,0,53.1000,1,S
4,3,0,3.5,0,0,8.0500,0,S
...,...,...,...,...,...,...,...,...
886,2,0,2.7,0,0,13.0000,0,S
887,1,1,1.9,0,0,30.0000,1,S
888,3,1,2.8,1,2,23.4500,0,S
889,1,0,2.6,0,0,30.0000,1,C


<br>

## 2. Train - Test split (비율을 7:3 으로 유지해주시고, seed는 0을 적용해주세요)

In [ ]:
from sklearn import datasets, model_selection, linear_model

In [12]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.3, random_state=0)


NameError: name 'model_selection' is not defined

In [13]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

NameError: name 'x_train' is not defined

In [14]:
model = linear_model.LogisticRegression() # 로지스틱회귀

NameError: name 'linear_model' is not defined

In [207]:
model.fit(x_train, y_train)

ValueError: could not convert string to float: 'S'

In [ ]:
x_test[:]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin
495,3,0,2.8,0,0,14.4583,0
648,3,0,2.8,0,0,7.5500,0
278,3,0,0.7,4,1,29.1250,0
31,1,1,2.8,1,0,146.5208,1
255,3,1,2.9,0,2,15.2458,0
...,...,...,...,...,...,...,...
263,1,0,4.0,0,0,0.0000,1
718,3,0,2.8,0,0,15.5000,0
620,3,0,2.7,1,0,14.4542,0
786,3,1,1.8,0,0,7.4958,0


In [ ]:
from sklearn.metrics import accuracy_score # accuracy

print('Accuracy: ', accuracy_score(model.predict(x_test), y_test))

Accuracy:  0.8171641791044776


<br>

## 3. Create model instance variable (동시에 여러 모델을 다른 이름으로 만들 수 있습니다.)

<br>

## 4. Train the model

<br>

## 5. Predict on test data & Check the result with metrics (모델 간 비교가 가능합니다.)